# Weather Agent

In [122]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [123]:
# Cell 1: Imports & constants
# --------------------------------
# - Basic imports
# - Set hourly/daily variable names to request from Open-Meteo (the 7 chosen ones)
# - Fallback timezone offsets (used if system lacks zoneinfo/tzdata)
import requests
from datetime import datetime, timedelta
import math

try:
    from zoneinfo import ZoneInfo  # Python 3.9+; recommended
except Exception:
    ZoneInfo = None

# Open-Meteo variables we will request (minimal farming set)
HOURLY_VARS = [
    "temperature_2m",
    "relativehumidity_2m",
    "dewpoint_2m",
    "precipitation",
    "windspeed_10m",
    "shortwave_radiation",
    "vapour_pressure_deficit"  # optional if API supports, else computed
]

DAILY_VARS = [
    "temperature_2m_max",
    "temperature_2m_min",
    "precipitation_sum",
    "shortwave_radiation_sum"
]

# fallback timezone offsets in hours (for environments without zoneinfo)
TZ_OFFSET_FALLBACK = {"Asia/Kolkata": 5.5, "UTC": 0.0}


Sub-agent: GeocodingAgent

In [124]:
# Cell 2: GeocodingAgent
# --------------------------------
# Function: geocode_location(location)
# - Calls Open-Meteo geocoding API and returns a dict with lat/lon/timezone and a nice name.
# - Input: free-text location (village/city/district)
# - Output: dict or None
def geocode_location(location: str):
    url = "https://geocoding-api.open-meteo.com/v1/search"
    params = {"name": location, "count": 1, "language": "en", "format": "json"}
    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    if "results" in data and len(data["results"]) > 0:
        res = data["results"][0]
        return {
            "name": res.get("name"),
            "latitude": res.get("latitude"),
            "longitude": res.get("longitude"),
            "country": res.get("country"),
            "admin1": res.get("admin1"),
            "timezone": res.get("timezone")
        }
    return None


ForecastFetchAgent

In [126]:
# Cell 3: ForecastFetchAgent
# --------------------------------
# Function: fetch_open_meteo(lat, lon, timezone_name, days=7)
# - Fetches hourly + daily variables from Open-Meteo for next `days`.
# - Uses HOURLY_VARS and DAILY_VARS defined above.
def fetch_open_meteo(lat, lon, timezone_name, days=7):
    start_date = datetime.utcnow().date()
    end_date = start_date + timedelta(days=days)
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "timezone": timezone_name,
        "hourly": ",".join(HOURLY_VARS),
        "daily": ",".join(DAILY_VARS),
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat()
    }
    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    return r.json()


In [127]:
raw = fetch_open_meteo(para["latitude"], para["longitude"], para.get("timezone") or "UTC", days=7)
print(raw)

C:\Users\vanshita.vani\AppData\Local\Temp\ipykernel_4628\1481835959.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_date = datetime.utcnow().date()


{'latitude': 22.625, 'longitude': 75.25, 'generationtime_ms': 0.8013248443603516, 'utc_offset_seconds': 19800, 'timezone': 'Asia/Kolkata', 'timezone_abbreviation': 'GMT+5:30', 'elevation': 560.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relativehumidity_2m': '%', 'dewpoint_2m': '°C', 'precipitation': 'mm', 'windspeed_10m': 'km/h', 'shortwave_radiation': 'W/m²', 'vapour_pressure_deficit': 'kPa'}, 'hourly': {'time': ['2025-12-03T00:00', '2025-12-03T01:00', '2025-12-03T02:00', '2025-12-03T03:00', '2025-12-03T04:00', '2025-12-03T05:00', '2025-12-03T06:00', '2025-12-03T07:00', '2025-12-03T08:00', '2025-12-03T09:00', '2025-12-03T10:00', '2025-12-03T11:00', '2025-12-03T12:00', '2025-12-03T13:00', '2025-12-03T14:00', '2025-12-03T15:00', '2025-12-03T16:00', '2025-12-03T17:00', '2025-12-03T18:00', '2025-12-03T19:00', '2025-12-03T20:00', '2025-12-03T21:00', '2025-12-03T22:00', '2025-12-03T23:00', '2025-12-04T00:00', '2025-12-04T01:00', '2025-12-04T02:00', '2025-12-04T03:00', '

 QualityCheckAgent (light)

In [129]:
def quality_check(raw_json):
    hourly = raw_json.get("hourly", {})
    daily = raw_json.get("daily", {})
    # basic alignment check - ensure arrays exist
    # return cleaned dict with same keys (caller will test availability)
    return {"hourly": hourly, "daily": daily}

Get current weather 

In [130]:
from datetime import datetime
try:
    # prefer zoneinfo when available (Python 3.9+)
    from zoneinfo import ZoneInfo
except Exception:
    ZoneInfo = None



def pick_closest_hourly_sample(hourly: dict):
    """
    100% reliable: Picks hourly value closest to current IST hour.
    This does NOT use timezone objects, so no ZoneInfo issues.
    Open-Meteo hourly times are already in local timezone.
    """
    times = hourly.get("time", [])
    temps = hourly.get("temperature_2m", [])

    if not times or not temps:
        return None

    # Current IST hour
    now_hour = datetime.now().hour  # system hour (IST on Windows)

    best_idx = 0
    best_diff = 999

    for i, tstr in enumerate(times):
        try:
            hour = int(tstr.split("T")[1].split(":")[0])
        except:
            continue

        diff = abs(hour - now_hour)
        if diff < best_diff:
            best_diff = diff
            best_idx = i

    return {
        "time": times[best_idx],
        "temp": temps[best_idx]
    }


Minimal metrics extractor

In [131]:
# Cell 5: Minimal metrics extractor (daily means and 7-day precipitation)
# --------------------------------
# - Extracts the daily arrays and computes:
#   * precip_7d_total
#   * daily_time, daily_tmin/tmax/precip (for table)
# - Optionally compute daily mean RH, mean wind, mean shortwave from hourly arrays (if present)
def minimal_metrics_from_raw(raw):
    daily = raw.get("daily", {})
    hourly = raw.get("hourly", {})

    d_time = daily.get("time", [])
    d_tmin = daily.get("temperature_2m_min", [])
    d_tmax = daily.get("temperature_2m_max", [])
    d_precip = daily.get("precipitation_sum", [])
    d_swr = daily.get("shortwave_radiation_sum", [])

    # compute 7-day precip total (sum of available daily_precip)
    precip7 = None
    if d_precip:
        precip7 = sum([p for p in d_precip if p is not None])

    # compute daily mean RH / wind / shortwave from hourly (optional)
    h_time = hourly.get("time", [])
    h_rh = hourly.get("relativehumidity_2m", [])
    h_wind = hourly.get("windspeed_10m", [])
    h_swr = hourly.get("shortwave_radiation", [])

    # helper to compute daily means from hourly arrays (align by date)
    def hourly_daily_means(h_times, h_vals):
        if not h_times or not h_vals:
            return {}
        day_map = {}
        for ts, v in zip(h_times, h_vals):
            if v is None:
                continue
            date = ts.split("T")[0]
            day_map.setdefault(date, []).append(v)
        return {d: (sum(vals)/len(vals)) for d, vals in day_map.items()}

    rh_mean_by_day = hourly_daily_means(h_time, h_rh)
    wind_mean_by_day = hourly_daily_means(h_time, h_wind)
    swr_mean_by_day = hourly_daily_means(h_time, h_swr)

    return {
        "daily_time": d_time,
        "daily_tmin": d_tmin,
        "daily_tmax": d_tmax,
        "daily_precip": d_precip,
        "daily_swr_sum": d_swr,
        "precip_7d_total": precip7,
        "rh_mean_by_day": rh_mean_by_day,
        "wind_mean_by_day": wind_mean_by_day,
        "swr_mean_by_day": swr_mean_by_day,
        "hourly": hourly
    }


FormatterAgent (labeled)

In [132]:
# Cell 6: Formatter for the compact 7-day table (prints only essential fields)
# --------------------------------
# - Builds a short plain-text report:
#   Location, Report time, Current temp, 7-day precip total & availability,
#   and a compact day-by-day table with Tmin/Tmax/Precip/MeanRH/MeanWind/Shortwave/ET0(optional)
# - We include MeanRH and MeanWind only if computed above.
def format_compact_table(geo, metrics, sample):
    lines = []
    loc = geo.get("name") or ""
    if geo.get("admin1"): loc += f", {geo.get('admin1')}"
    if geo.get("country"): loc += f", {geo.get('country')}"
    lines.append(f"Location: {loc}")
    lines.append(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    if sample and sample.get("temp") is not None:
        lines.append(f"Current temperature (latest): {sample['temp']:.1f} °C (as of {sample['time']})")
    if metrics.get("precip_7d_total") is not None:
        p7 = metrics["precip_7d_total"]
        avail = "High" if p7 >= 50 else ("Medium" if p7 >= 15 else "Low")
        lines.append(f"Rainfall (next 7 days total): {p7:.1f} mm → Water availability: {avail}")
    lines.append("")  # separator
    lines.append("7-day compact table (Date | Tmin/Tmax °C | Precip mm | MeanRH% | Wind m/s | Shortwave sum):")
    for d, tmin, tmax, p in zip(metrics.get("daily_time", []), metrics.get("daily_tmin", []), metrics.get("daily_tmax", []), metrics.get("daily_precip", [])):
        parts = [f"{d}"]
        if tmin is not None and tmax is not None:
            parts.append(f"{tmin:.1f}/{tmax:.1f}°C")
        else:
            parts.append("-")
        parts.append(f"{p:.1f}" if p is not None else "-")
        # mean RH/wind/swr if available
        rh = metrics.get("rh_mean_by_day", {}).get(d)
        wind = metrics.get("wind_mean_by_day", {}).get(d)
        swr_sum = metrics.get("daily_swr_sum", [])
        swr_val = None
        if metrics.get("daily_swr_sum"):
            # daily_swr_sum aligns with daily_time list; find index
            try:
                idx = metrics["daily_time"].index(d)
                swr_val = metrics.get("daily_swr_sum")[idx]
            except Exception:
                swr_val = None
        parts.append(f"{rh:.0f}%" if rh is not None else "-")
        parts.append(f"{wind:.1f}" if wind is not None else "-")
        parts.append(f"{swr_val:.1f}" if swr_val is not None else "-")
        lines.append(" - " + " | ".join(parts))
    # confidence
    conf = 0.80 if metrics.get("precip_7d_total") is not None else 0.60
    lines.append(f"\nConfidence: {conf:.2f}")
    return "\n".join(lines)


weather_agent_location (public)

In [133]:
# Cell 7: Orchestrator - minimal 7-day fetch + format
# --------------------------------
# - This cell shows how to call the above pieces together.
# - Input: free-text location string.
# - Output: plain-text compact 7-day table (printed).
def weather_7day_compact(location: str, days: int = 7):
    geo = geocode_location(location)
    if not geo:
        return f"Location: {location}\nError: could not geocode."
    raw = fetch_open_meteo(geo["latitude"], geo["longitude"], geo.get("timezone") or "UTC", days=days)
    cleaned = {"daily": raw.get("daily", {}), "hourly": raw.get("hourly", {})}
    metrics = minimal_metrics_from_raw(cleaned)
    sample = pick_closest_hourly_sample(cleaned.get("hourly", {}))
    report_text = format_compact_table(geo, metrics, sample)
    return report_text

# Example usage (uncomment to run)
# print(weather_7day_compact("Dhar, Madhya Pradesh", days=7))


Testing

In [134]:
report = weather_7day_compact("Dhar", days=7)
print("----------------------weather data report-----------------------")
print(report)

C:\Users\vanshita.vani\AppData\Local\Temp\ipykernel_4628\1481835959.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_date = datetime.utcnow().date()


----------------------weather data report-----------------------
Location: Dhār, Madhya Pradesh, India
Report generated: 2025-12-03 15:52
Current temperature (latest): 25.9 °C (as of 2025-12-03T15:00)
Rainfall (next 7 days total): 0.0 mm → Water availability: Low

7-day compact table (Date | Tmin/Tmax °C | Precip mm | MeanRH% | Wind m/s | Shortwave sum):
 - 2025-12-03 | 14.0/26.0°C | 0.0 | 62% | 8.0 | 17.3
 - 2025-12-04 | 13.0/26.1°C | 0.0 | 62% | 7.9 | 17.4
 - 2025-12-05 | 13.2/26.2°C | 0.0 | 62% | 9.2 | 17.5
 - 2025-12-06 | 11.9/26.3°C | 0.0 | 56% | 9.5 | 17.7
 - 2025-12-07 | 10.9/27.5°C | 0.0 | 52% | 7.3 | 17.7
 - 2025-12-08 | 11.9/27.6°C | 0.0 | 54% | 6.9 | 17.6
 - 2025-12-09 | 11.9/28.1°C | 0.0 | 54% | 5.6 | 17.7
 - 2025-12-10 | 11.7/27.3°C | 0.0 | 46% | 4.1 | 17.7

Confidence: 0.80
